In [103]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
%matplotlib widget

In [104]:
base_path = Path('../Data/100610')

In [105]:
def get_v_path(number, base_path):
    return base_path / f'dyads{number}.nii.gz'
def get_f_path(number, base_path):
    return base_path / f'mean_f{number}samples.nii.gz'

In [106]:
vector_imgs = [nib.load(get_v_path(i+1, base_path)).get_fdata() for i in range(3)]
fraction_imgs = [nib.load(get_f_path(i+1, base_path)).get_fdata() for i in range(3)]

In [121]:
def plot_slice(z_slice, fibers, mask_threshold):
    """
    Affiche la slice z_slice.
    fibers: liste des fibres à afficher, ex: [1] ou [1, 2]
    """

    slice_x, slice_y = slice(0, 145), slice(0, 145)

    plt.figure(figsize=(6, 6), facecolor='white')
    ax = plt.gca()
    ax.set_facecolor('black')

    dims = fraction_imgs[0][slice_x, slice_y, z_slice].shape
    x_grid, y_grid = np.meshgrid(np.arange(dims[0]), np.arange(dims[1]), indexing='ij')

    colors = {1: 'red', 2: 'cyan', 3: 'yellow'}

    for fib_idx in fibers:
        if fib_idx > len(vector_imgs): continue 

        vec_data = vector_imgs[fib_idx - 1][slice_x, slice_y, z_slice, :]
        current_mask = fraction_imgs[fib_idx - 1][slice_x, slice_y, z_slice] > mask_threshold
        
        plt.quiver(x_grid[current_mask], y_grid[current_mask],
                   vec_data[current_mask, 0], vec_data[current_mask, 1],
                   color=colors.get(fib_idx, 'white'),
                   scale=30, headwidth=0, headlength=0, width=0.005, alpha=0.9,
                   label=f'Fibre {fib_idx}')

    plt.title(f"Slice Z={z_slice} | Fibres {fibers}", color='white')

    plt.legend(loc='upper right', fontsize='small')
    plt.axis('off')
    plt.show()

In [127]:
interact(
    plot_slice,
    z_slice=widgets.IntSlider(min=0, max=img_v1.shape[2]-1, step=1, value=img_v1.shape[2]//2, continuous_update=False),
    mask_threshold=widgets.FloatSlider(min=0, max=1, step=0.01, value=0.1, continuous_update=False),    
    fibers=widgets.SelectMultiple(options=[1, 2, 3], value=[1], description='fibers:'),
)

interactive(children=(IntSlider(value=72, continuous_update=False, description='z_slice', max=144), SelectMult…

<function __main__.plot_slice(z_slice, fibers, mask_threshold)>

In [116]:
brain_vector_field = vector_imgs[0][:, :, 72]
brain_mask = fraction_imgs[0][:, :, 72] > 0.1